In [1]:
# Okay, since we did not get the model to train on the nb, let's fit the NNe in a separate script and try to inlcude this in the python script.

In [2]:
import os
import json
import sys
import warnings
from pathlib import Path
from pprint import pformat
from typing import Dict, Union
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import pickle
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from create_data_generator import data_generator, batch_predict

# [Req] IMPROVE imports
# notice that the improvelibs are in the folder that is a level above, but in the same parent directory
sys.path.append(os.path.abspath(os.path.join('..', 'IMPROVE')))
from improvelib.applications.drug_response_prediction.config import DRPTrainConfig
from improvelib.utils import str2bool
import improvelib.utils as frm
from improvelib.metrics import compute_metrics

# Model-specific imports
from model_params_def import train_params # [Req]

2025-04-10 10:53:07.101446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 10:53:12.593581: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
data_dir = 'exp_result'

In [4]:
%%time
# load the models
cancer_gen_expr_model = tf.keras.models.load_model(os.path.join(data_dir,"cancer_gen_expr_model"))
cancer_gen_mut_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_gen_mut_model"))
cancer_dna_methy_model = tf.keras.models.load_model(os.path.join(data_dir, "cancer_dna_methy_model"))

2025-04-10 10:54:42.293902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 10:54:49.732819: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30960 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


WARNING tensorflow 2025-04-10 10:54:51,164:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-10 10:54:51,511:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


WARNING tensorflow 2025-04-10 10:54:51,870:	No training configuration found in save file, so the model was *not* compiled. Compile it manually.


CPU times: user 1.44 s, sys: 1.02 s, total: 2.46 s
Wall time: 10 s


In [5]:
cancer_gen_expr_model.trainable = False
cancer_gen_mut_model.trainable = False
cancer_dna_methy_model.trainable = False

In [6]:
with open(os.path.join(data_dir, "drug_features.pickle"),"rb") as f:
        dict_features = pickle.load(f)

In [7]:
with open(os.path.join(data_dir, "norm_adj_mat.pickle"),"rb") as f:
        dict_adj_mat = pickle.load(f)

In [8]:
train_keep = pd.read_csv(os.path.join(data_dir, "train_y_data.csv"))
valid_keep = pd.read_csv(os.path.join(data_dir, "val_y_data.csv"))

In [9]:
train_keep.head()

,improve_sample_id,improve_chem_id,auc
0,ACH-000956,Drug_749,0.7153
1,ACH-000956,Drug_1326,0.9579
2,ACH-000956,Drug_490,0.4130
3,ACH-000956,Drug_558,0.8004
4,ACH-000956,Drug_195,0.5743


In [10]:
valid_keep.head()

,improve_sample_id,improve_chem_id,auc
0,ACH-000956,Drug_895,0.5071
1,ACH-000956,Drug_614,0.6525
2,ACH-000956,Drug_1040,0.8944
3,ACH-000323,Drug_1279,0.8691
4,ACH-000323,Drug_685,0.8527


In [11]:
train_keep.shape, valid_keep.shape

((7616, 3), (952, 3))

In [12]:
train_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]
valid_keep.columns = ["Cell_Line", "Drug_ID", "AUC"]

In [13]:
train_keep.head()

,Cell_Line,Drug_ID,AUC
0,ACH-000956,Drug_749,0.7153
1,ACH-000956,Drug_1326,0.9579
2,ACH-000956,Drug_490,0.4130
3,ACH-000956,Drug_558,0.8004
4,ACH-000956,Drug_195,0.5743


In [14]:
valid_keep.head()

,Cell_Line,Drug_ID,AUC
0,ACH-000956,Drug_895,0.5071
1,ACH-000956,Drug_614,0.6525
2,ACH-000956,Drug_1040,0.8944
3,ACH-000323,Drug_1279,0.8691
4,ACH-000323,Drug_685,0.8527


In [15]:
samp_drug = valid_keep["Drug_ID"].unique()[-1]
samp_ach = np.array(valid_keep["Cell_Line"].unique()[-1])

In [16]:
print(samp_drug)
print(samp_ach)

Drug_1326
ACH-000828


In [17]:
train_gcn_feats = []
train_adj_list = []
for drug_id in train_keep["Drug_ID"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_adj_mat[drug_id])

In [18]:
len(train_gcn_feats), len(train_adj_list)

(7616, 7616)

In [19]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in valid_keep["Drug_ID"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_adj_mat[drug_id])

In [20]:
len(valid_gcn_feats), len(valid_adj_list)

(952, 952)

In [21]:
len(valid_gcn_feats) + len(train_gcn_feats)

8568

In [22]:
%%time
# reduce the values to float16
train_gcn_feats = np.array(train_gcn_feats).astype("float32")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float32")

train_adj_list = np.array(train_adj_list).astype("float32")
valid_adj_list = np.array(valid_adj_list).astype("float32")

CPU times: user 561 ms, sys: 768 ms, total: 1.33 s
Wall time: 1.33 s


In [23]:
# load trained model here
CCLE_model = tf.keras.models.load_model('exp_result/DeepCDR_model/DeepCDR_model')

In [24]:
CCLE_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 223, 223)]   0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 223, 75)]    0           []                               
                                                                                                  
 dot (Dot)                      (None, 223, 75)      0           ['input_2[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 223, 256)     19456       ['dot[0][0]']                

In [25]:
# create a data generator for the train data
batch_size = 32
generator_batch_size = 32

In [26]:
# # get the predictions on the validation set

val_steps = int(np.ceil(len(valid_gcn_feats) / generator_batch_size))
preds_val, target_val = batch_predict(CCLE_model, data_generator(valid_gcn_feats, valid_adj_list, valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["Cell_Line"].values.reshape(-1,1), valid_keep["AUC"].values.reshape(-1,1), generator_batch_size, shuffle = False), val_steps)   

Generating batch from index 0 to 32


2025-04-10 10:55:09.433100: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


Generating batch from index 32 to 64
Generating batch from index 64 to 96
Generating batch from index 96 to 128
Generating batch from index 128 to 160
Generating batch from index 160 to 192
Generating batch from index 192 to 224
Generating batch from index 224 to 256
Generating batch from index 256 to 288
Generating batch from index 288 to 320
Generating batch from index 320 to 352
Generating batch from index 352 to 384
Generating batch from index 384 to 416
Generating batch from index 416 to 448
Generating batch from index 448 to 480
Generating batch from index 480 to 512
Generating batch from index 512 to 544
Generating batch from index 544 to 576
Generating batch from index 576 to 608
Generating batch from index 608 to 640
Generating batch from index 640 to 672
Generating batch from index 672 to 704
Generating batch from index 704 to 736
Generating batch from index 736 to 768
Generating batch from index 768 to 800
Generating batch from index 800 to 832
Generating batch from index 83

In [27]:
frm.store_predictions_df(
        y_true=target_val, 
        y_pred=preds_val, 
        stage="val",
        y_col_name="auc",
        output_dir='new_result',
        input_dir='exp_result'
    )

In [30]:
val_scores = frm.compute_performance_scores(
        y_true=target_val, 
        y_pred=preds_val, 
        stage="val",
        metric_type='regression',
        output_dir= 'new_result'
    )


IMPROVE_RESULT val_loss:	0.005228987737754953

Validation scores:
	{'mse': 0.005228987737754953, 'rmse': 0.07231173997183965, 'pcc': 0.886184480102248, 'scc': 0.7734529884496383, 'r2': 0.780726142084764}


In [31]:
# Predict the values for train set as well, because we need these for the NNE

In [33]:
train_steps = int(np.ceil(len(train_gcn_feats) / generator_batch_size))
preds_train, target_train = batch_predict(CCLE_model, data_generator(train_gcn_feats, train_adj_list, train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["Cell_Line"].values.reshape(-1,1), train_keep["AUC"].values.reshape(-1,1), generator_batch_size, shuffle = False), train_steps) 

Generating batch from index 0 to 32
Generating batch from index 32 to 64
Generating batch from index 64 to 96
Generating batch from index 96 to 128
Generating batch from index 128 to 160
Generating batch from index 160 to 192
Generating batch from index 192 to 224
Generating batch from index 224 to 256
Generating batch from index 256 to 288
Generating batch from index 288 to 320
Generating batch from index 320 to 352
Generating batch from index 352 to 384
Generating batch from index 384 to 416
Generating batch from index 416 to 448
Generating batch from index 448 to 480
Generating batch from index 480 to 512
Generating batch from index 512 to 544
Generating batch from index 544 to 576
Generating batch from index 576 to 608
Generating batch from index 608 to 640
Generating batch from index 640 to 672
Generating batch from index 672 to 704
Generating batch from index 704 to 736
Generating batch from index 736 to 768
Generating batch from index 768 to 800
Generating batch from index 800 t

In [34]:
frm.store_predictions_df(
        y_true=target_train, 
        y_pred=preds_train, 
        stage="train",
        y_col_name="auc",
        output_dir='new_result',
        input_dir='exp_result'
    )

In [35]:
val_scores = frm.compute_performance_scores(
        y_true=target_train, 
        y_pred=preds_train, 
        stage="train",
        metric_type='regression',
        output_dir= 'new_result'
    )

Invalid stage: must be 'val' or 'test'.


In [37]:
# Notice this still creates the json file for the train scores even if it spits out that the stage should be val or test, so evrything is okay. 

In [36]:
# So what next? We still need to be able to able ot train the models for a number of iterations inorder to get the bootsrap predictions

In [38]:
# Does not seem that straight forward

In [39]:
# Should we have whatever we have in a .py script from the trained failed jupyternotebook and see if this script works? That way, it might be easier to debug